In [1]:
import numpy as np
import pandas as pd
import nltk
import unicodedata
import re
import gensim.corpora as corpora
import tensorflow as tf
import spacy
import string
import scipy
import gensim

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from nltk.corpus import wordnet as wn,stopwords
from nltk import CFG,word_tokenize
from nltk.stem import PorterStemmer,LancasterStemmer,WordNetLemmatizer
from nltk.parse.generate import generate
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

from gensim.models import Word2Vec
from textblob import TextBlob,Word
from spacy import displacy
from bs4 import BeautifulSoup
from afinn import Afinn

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Bidirectional,Embedding,SpatialDropout1D,Flatten,Reshape
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers ,regularizers

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Load the required data

In [2]:
raw=pd.read_csv('Raw_Skills_Dataset.csv')
tech=pd.read_csv('Example_Technical_Skills.csv')

In [3]:
raw.head()

,RAW DATA
0,What ifs
1,seniority
2,familiarity
3,functionalities
4,Lambdas


In [4]:
tech.head()

,Technology Skills
0,SAP Fiori Developer
1,Oracle Instance Management & Strategy
2,Boomi Master Data Management
3,Digital Manufacturing on Cloud ( DMC)
4,DevOps


In [5]:
print(raw.shape,tech.shape)

(34116, 1) (979, 1)


Approach Used:

1. Gather the sentence vector form for technical data
2. Gather the sentence vector form for raw data
3. For every skill vector in raw data find the cosine similarity btw it and the tech data and if its above the threshold mark it hard skill 

Approach with sentence transformers

In [42]:
pip install -U sentence-transformers

     ---------------------------------------- 79.7/79.7 KB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 4.2/4.2 MB 33.2 MB/s eta 0:00:00
     ------------------------------------- 158.0/158.0 MB 10.9 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 32.5 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 17.2 MB/s eta 0:00:00
     ---------------------------------------- 86.2/86.2 KB ? eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 26.2 MB/s eta 0:00:00
     ---------------------------------------- 40.8/40.8 KB 1.9 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\prana\anaconda3\python.exe -m pip install --upgrade pip' command.



  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120753 sha256=9161059e3a8217d4cb2aca03860948c5913127e158780b83ead3c1ae0bd91504
  Stored in directory: c:\users\prana\appdata\local\pip\cache\wheels\0c\b6\fb\2289a932c365293ad865fc1fe9d2db694d5584241c6d670874
Successfully built sentence-transformers
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4


In [43]:
from sentence_transformers import SentenceTransformer
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [44]:
from scipy.spatial import distance
raw_skills=raw['RAW DATA']
tech_skills=tech['Technology Skills']

In [49]:
raw_embeddings=model.encode(raw_skills)

In [51]:
tech_embeddings=model.encode(tech_skills)

In [69]:
check_list=[]
for i in raw_embeddings:
    listed=[]
    for j in tech_embeddings[0:1500]:
        ans=1 - distance.cosine(i,j)
        if ans>0.5:
            listed.append('yes')
        else:
            listed.append('no')
    check_list.append(max(set(listed),key=listed.count))

In [70]:
check_list[0:25]


['no',
 'no',
 'no',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'yes',
 'no',
 'no',
 'yes',
 'no',
 'no',
 'yes',
 'yes']

In [71]:
raw['is_it_hard_skill']=pd.Series(check_list)

In [72]:
raw.head()

,RAW DATA,is_it_hard_skill
0,What ifs,no
1,seniority,no
2,familiarity,no
3,functionalities,yes
4,Lambdas,yes


In [73]:
raw['is_it_hard_skill'].value_counts()

yes    20892
no     13224
Name: is_it_hard_skill, dtype: int64

Let's see the hard skills detected

In [78]:
print(raw.loc[raw['is_it_hard_skill']=='yes'][450:500])

                                   RAW DATA is_it_hard_skill
693                     the project roadmap              yes
694                   emerging technologies              yes
695                              frameworks              yes
696                  cloud-based ecosystems              yes
698                                 scripts              yes
700                 production environments              yes
702                                     Vue              yes
703                                 .. APIs              yes
704                     emerging frameworks              yes
706                                    Node              yes
707                              Typescript              yes
711                                     GCP              yes
712                Operational efficiencies              yes
721                                     API              yes
722                           UI components              yes
723                     